## run request with owslib

In [1]:
from owslib.wps import WebProcessingService, monitorExecution, printInputOutput


In [2]:
wps_url = "http://localhost:8093/wps"
#wps_url = "http://birdhouse-lsce.extra.cea.fr:8093/wps"
wps = WebProcessingService(url=wps_url, verbose=False)

In [3]:
for process in wps.processes:
    print '%s : \t %s' % (process.identifier, process.abstract, )

visualisation : 	 Just testing a nice script to visualise some variables
extractpoints : 	 Extract Timeseries for specified coordinates from grid data
segetalflora : 	 Species biodiversity of segetal flora. Imput files: variable:tas , domain: EUR-11 or EUR-44
indices_single : 	 This process calculates climate indices based on one single variable.
subset_countries : 	 This process returns only the given polygon from input netCDF files.
eobs_to_cordex : 	 downloades EOBS data in adaped CORDEX format
ensembleRobustness : 	 Calculates the robustness as the ratio of noise to signal in an ensemle of timeseries


In [12]:
in_file1 = 'file:///home/estimr1/EUCLEIA/indices/RX5day/DJF/RX5day_DJF_HadGEM3-A-N216_historical_r1i1p1_19600101-20131230.nc'
in_file2 = 'file:///home/estimr1/EUCLEIA/indices/RX5day/DJF/RX5day_DJF_HadGEM3-A-N216_historical_r1i1p2_19600101-20131230.nc'

#in_file1 = 'file:///homel/nhempel/data/NCEP/slp.1948-2014_NA.nc'
#in_file1 ='file:///home/estimr2/yiou/NCEP/slp.1982_NHX.nc'
#in_file1 = 'file:///home/pingu/.conda/envs/birdhouse/var/lib/cache/pywps/esgf1.dkrz.de/thredds/fileServer/cordex/cordex/output/EUR-44/MPI-CSC/MPI-M-MPI-ESM-LR/historical/r1i1p1/MPI-CSC-REMO2009/v1/mon/tasmax/v20150609/tasmax_EUR-44_MPI-M-MPI-ESM-LR_historical_r1i1p1_MPI-CSC-REMO2009_v1_mon_199101-200012.nc'
#in_file2 = 'file:///home/pingu/.conda/envs/birdhouse/var/lib/cache/pywps/esgf1.dkrz.de/thredds/fileServer/cordex/cordex/output/EUR-44/MPI-CSC/MPI-M-MPI-ESM-LR/historical/r1i1p1/MPI-CSC-REMO2009/v1/mon/tasmax/v20150609/tasmax_EUR-44_MPI-M-MPI-ESM-LR_historical_r1i1p1_MPI-CSC-REMO2009_v1_mon_200101-200512.nc'

In [ ]:
execute = wps.execute(
    identifier="subset_countries", #indices_clipping",
    inputs=[
    ("resource",in_file1),
    ("resource",in_file2),
    ("region",'FRA'),
      # ("country",'DEU'),
    ("mosaik",'False'),
           ])

monitorExecution(execute, sleepSecs=5)

print execute.getStatus()

for o in execute.processOutputs:
    print o.reference
    

## run sync request with get parameters

In [13]:
sync_req_url = "{wps_url}?" +\
    "request=Execute" +\
    "&service=WPS" +\
    "&version=1.0.0" +\
    "&identifier=subset_countries" +\
    "&DataInputs=resource={resource1};resource={resource2};region={region}" +\
    "&storeExecuteResponse=false" +\
    "&status=false"

In [14]:
url=sync_req_url.format(
    wps_url=wps_url,
    resource1=in_file1,
    resource2=in_file2,
    region='FRA')
print url   

http://localhost:8093/wps?request=Execute&service=WPS&version=1.0.0&identifier=subset_countries&DataInputs=resource=file:///home/estimr1/EUCLEIA/indices/RX5day/DJF/RX5day_DJF_HadGEM3-A-N216_historical_r1i1p1_19600101-20131230.nc;resource=file:///home/estimr1/EUCLEIA/indices/RX5day/DJF/RX5day_DJF_HadGEM3-A-N216_historical_r1i1p2_19600101-20131230.nc;region=FRA&storeExecuteResponse=false&status=false


In [15]:
import requests
r = requests.get(url)
print r.status_code
print r.text

502
<html>
<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.8.0</center>
</body>
</html>



## run async request with get parameters

In [17]:
async_req_url = "{wps_url}?" +\
    "request=Execute" +\
    "&service=WPS" +\
    "&version=1.0.0" +\
    "&identifier=subset_countries" +\
    "&DataInputs=resource={resource1};resource={resource2};region={region}" +\
    "&storeExecuteResponse=true" +\
    "&status=true"
    #

In [18]:
url=async_req_url.format(
    wps_url=wps_url,
    resource1=in_file1,
    resource2=in_file2,
    region='FRA')
print url   

http://localhost:8093/wps?request=Execute&service=WPS&version=1.0.0&identifier=subset_countries&DataInputs=resource=file:///home/estimr1/EUCLEIA/indices/RX5day/DJF/RX5day_DJF_HadGEM3-A-N216_historical_r1i1p1_19600101-20131230.nc;resource=file:///home/estimr1/EUCLEIA/indices/RX5day/DJF/RX5day_DJF_HadGEM3-A-N216_historical_r1i1p2_19600101-20131230.nc;region=FRA&storeExecuteResponse=true&status=true


In [19]:
r = requests.get(url)
from lxml import etree
from io import BytesIO
tree = etree.parse(BytesIO(r.content))
#print etree.tostring(tree)
status_url = tree.getroot().get("statusLocation")
print status_url

http://localhost:8090/wpsoutputs/flyingpigeon/pywps-010321ac-bf87-11e5-807f-c7a428fab5b1.xml


In [10]:
r = requests.get(url)
from lxml import etree
from io import BytesIO
tree = etree.parse(BytesIO(r.content))
#print etree.tostring(tree)
status_url = tree.getroot().get("statusLocation")
print status_url

http://localhost:8090/wpsoutputs/flyingpigeon/pywps-2f2adf30-bf86-11e5-9605-770c8ca47feb.xml


In [ ]:
r = requests.get(status_url)
print r.status_code
print r.text

In [20]:
status_url2 = status_url

In [23]:
r = requests.get(status_url2)
print r.status_code
print r.text

200
<?xml version="1.0" encoding="utf-8"?>
<wps:ExecuteResponse xmlns:wps="http://www.opengis.net/wps/1.0.0" xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/wps/1.0.0 http://schemas.opengis.net/wps/1.0.0/wpsExecute_response.xsd" service="WPS" version="1.0.0" xml:lang="en-CA" serviceInstance="http://localhost:8093/wps?service=WPS&amp;request=GetCapabilities&amp;version=1.0.0" statusLocation="http://localhost:8090/wpsoutputs/flyingpigeon/pywps-010321ac-bf87-11e5-807f-c7a428fab5b1.xml">
    <wps:Process wps:processVersion="0.1">
        <ows:Identifier>subset_countries</ows:Identifier>
        <ows:Title>Subset netCDF files</ows:Title>
        <ows:Abstract>This process returns only the given polygon from input netCDF files.</ows:Abstract>
    </wps:Process>
    <wps:Status creationTime="2016-01-20T16:06:01Z">
        <wps:ProcessSucceeded>PyWPS Process su